In [1]:
import os
import sys
import csv
import datetime
import shutil
import random
import glob
import pandas as pd
import numpy as np

In [29]:
# less eQTL.txt | grep -P "\trs" > eQTL.2.txt
# sed -i "1s/^/$(head -n1 eQTL.txt)\n/" eQTL.2.txt
in_file = "/home/galaxy/project/m6AQTL/data/pqtl/total_data/pQTL.txt"
df = pd.read_table(in_file, sep="\t", skiprows=1)  # , nrows=10 , skiprows=1
df["id"] = df["Ensembl_ID"] + ";" + df["Experimental_tag"].astype(
    str) + ";" + df["PubMed_ID"].astype(str)
# df["Gene_name"] = df.iloc[:, 3].str.split("|").str[0]
# df["Experimental_tag"] = df.iloc[:, 3].str.split("|").str.get(-2)
# df["PubMed_ID"] = df.iloc[:, 3].astype(str).str.split("|").str[-1]
# df["id"] = df["Gene_name"] + ";" + df["Experimental_tag"].astype(str) + ";" + df["PubMed_ID"].astype(str)
# ## 
# remove snp without a RS ID
df = df[df["RS_ID"].str.startswith("rs", na=False)]
# remove QTL without a Ensembl ID
df = df[df["id"].str.startswith("ENSG", na=False)]
print(df.head())

        Ensembl_ID Gene_Name Uniprot_ID Protein_Name         RS_ID  REF  ALT  \
0  ENSG00000120948    TARDBP        NaN       TARDBP  rs1471344782  NaN  NaN   
1  ENSG00000168610     STAT3        NaN       STAT3A   rs993990873  NaN  NaN   
2  ENSG00000100906    NFKBIA        NaN     IkBalpha  rs1477467102  NaN  NaN   
3  ENSG00000188342    GTF2F2        NaN       GTF2F2  rs1051246129  NaN  NaN   
4  ENSG00000270647     TAF15        NaN        TAF15  rs1452672552  NaN  NaN   

  pQTL_type Cutoff        Pvalue Corrected_pvalue Measure     Value  \
0     trans   1 Mb  3.156591e-08            < 0.2      R2  0.224095   
1       NaN    NaN  4.552463e-05              NaN       R  0.333490   
2       NaN    NaN  1.834424e-04              NaN       R -0.419501   
3       NaN    NaN  5.439675e-05              NaN       R  0.360772   
4       NaN    NaN  2.251364e-05              NaN       R -0.473694   

                     Cell_or_tissue     Condition  Experimental_tag PubMed_ID  \
0  lymphobl

In [5]:
result_dir = "/home/galaxy/project/m6AQTL/data/eqtl/total_data/"
# df_1 = df[df["Pvalue"].astype(str).str.replace(".","",1).str.replace("e","",1).str.replace("-","",1).str.isdigit()]
df_1 = df[df["Pvalue"].apply(lambda x: type(x) == float)]
print("\t" + str(len(df_1)))
df_result = df_1.groupby(["id"]).apply(pick_guard_snp)
# # print(df_result.head())
df_result.to_csv("%s/Guard_snp_from_paper.txt" % result_dir, sep="\t", index=False)

	2097133


In [4]:
def pick_guard_snp(df):
    df["Pvalue"] = df["Pvalue"].astype(float)
    df_sort = df.sort_values(["Pvalue"])
    return df_sort.iloc[0,:-1]

In [30]:
## filter by pvalue < CUTOFF
# raw_file = "/home/galaxy/project/m6AQTL/data/eqtl/total_data/Guard_snp_from_paper.txt"
# result_file = "/home/galaxy/project/m6AQTL/data/eqtl/total_data/Final_snp_from_paper.txt"
# df = pd.read_table(raw_file, sep="\t")
# df.head()
#### First version:
# Corrected_pvalue < 0.001 & Pvalue < 0.00001;
#### Second version:
# Corrected_pvalue < 0.05 & Pvalue < 0.00001;
#### Third version:
# Corrected_pvalue < 0.05 & Pvalue < 0.0001;
#### Fourth version:
#Corrected_pvalue < 0.001 & Pvalue < 0.00001;
result_file = "/home/galaxy/project/m6AQTL/data/pqtl/total_data/Final_snp.txt"
df_na = df[df["Pvalue"].isnull()]
df_na["Corrected_pvalue"] = df_na["Corrected_pvalue"].astype(float)
df_na = df_na[df_na["Corrected_pvalue"] < 0.001]
df_value = df[~df["Pvalue"].isnull()]
df_value = df_value[df_value["Pvalue"] < 0.00001]
df_result = pd.concat([df_na, df_value])
# df_result = df[df["Pvalue"] < 0.00000005] # 0.00001 or FDR < 0.001
# #
# df_result["Experimental_tag"] = df_result["Experimental_tag"].astype(int)
# df_result["PubMed_ID"] = df_result["PubMed_ID"].astype(int)
del df_result["id"]
df_result.to_csv(result_file, sep="\t", index=False)

/home/galaxy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
